In [93]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import pandas as pd
import glob
import os
from IPython.display import display
import matplotlib.pyplot as plt
import json
from sklearn import preprocessing
from keras.models import Model
from keras.layers import Input, LSTM, Dense, Embedding
from keras.preprocessing.text import Tokenizer
import tensorflow_addons as tfa
import decimal
from sklearn.preprocessing import normalize
from sklearn.preprocessing import MinMaxScaler
import math
from tensorflow.keras.layers.experimental.preprocessing import StringLookup

In [2]:
trailers_csv = "/Volumes/Seagate/natasha-diploma/trailers.csv"
trailers_df = pd.read_csv(trailers_csv, index_col=None, header=0)
trailers_df.drop(trailers_df.columns[trailers_df.columns.str.contains('Unnamed',case = False)],axis = 1, inplace = True)
display(trailers_df.head())

,title_x,duration,release_date,restrict,orig_title,kinopoisk_id,imdb_rating,kinopoisk_rating,synopsis,categories,...,emotions,Place of act,Qualities,Time of act,Based on,Audience,Mood,Subgenre,About,Theme
0,Философия Фила,5816.0,2019-04-05,18.0,Phil,960568,5.4,5.86,"Фил – стоматолог, страдающий от депрессии. Его...",{Фильмы},...,"{'neutral': 0.5312193632125854, 'negative': 0....",['США'],['дебютный'],"['21-й век', 'новейшее время']",['оригинального сценария'],"['для молодёжи', 'для взрослых']","['грустный', 'депрессивный', 'трогательный']","['Драмы', 'Комедийные']","['преследования', 'самозванцы', 'расследование...","['развлекательные', 'психологический']"
1,Зубная фея,5123.0,2019-04-02,18.0,Tooth Fairy,1126022,2.2,0.00,Старая женщина начинает рассказывать легенду о...,{Фильмы},...,"{'negative': 0.4688006341457367, 'skip': 0.392...",['не определено'],"['зрелищные', 'жанровый', 'неожиданный финал']",['не определено'],"['эпоса и легенд', 'оригинального сценария']","['для взрослых', 'для мужчин', 'для подготовле...","['трагичные', 'мрачные', 'шокирующие', 'захват...","['ужасы', 'триллер']","['преследования', 'страхи', 'дети', 'молодежь'...",['развлекательные']
2,Проснись,4667.0,2019-01-19,16.0,Wake Up,1069713,2.9,0.00,"Врач-психиатр исследует дневник девушки, котор...",{Фильмы},...,"{'positive': 0.3140605390071869, 'negative': 0...",['не определено'],"['жанровый', 'дебютный']",['новейшее время'],['оригинального сценария'],"['для взрослых', 'для подготовленного зрителя'...","['захватывающие', 'жестокие', 'напряженные', '...","['Драмы', 'Боевики', 'триллер']","['психотерапевты', 'кровь', 'тайна смерти', 'п...","['психологический', 'развлекательные']"
3,[4k] Адская кухня,6148.0,2019-08-09,16.0,The Kitchen,1117988,5.5,5.50,"Смотрите в 4К! Три жены ирландских гангстеров,...",{Фильмы},...,"{'neutral': 0.4765896201133728, 'negative': 0....","['большой город', 'США']","['жанровый', 'остросюжетные']",['20-й век'],['комиксов'],"['для мужчин', 'для женщин', 'для взрослых']","['захватывающие', 'авантюрный', 'жестокие', 'н...","['Боевики', 'Драмы', 'криминальные']","['враги или вражда', 'женщины', 'антигерои', '...","['авантюрные', 'гангстерская']"
4,[4k] Покемон. Детектив Пикачу,6013.0,2019-05-02,12.0,Pokémon Detective Pikachu,994864,6.6,6.56,Смотрите в 4K! Элитный детектив Пикачу отправл...,"{Для детей,Фильмы}",...,"{'neutral': 0.4532718360424042, 'negative': 0....","['вымышленная страна (город)', 'большой город']","['частично анимационный', 'зрелищные', 'кассов...",['21-й век'],"['оригинального сценария', 'сериалов']","['для детей 13-16 лет', '9 лет', '8 лет', '7-1...","['авантюрный', 'захватывающие', 'динамичный', ...","['детские', 'Приключенческие', 'Фантастические...","['притяжение противоположностей', 'риск', 'пре...","['развлекательные', 'образовательный']"


In [47]:
sessions_csv = "/Volumes/Seagate/natasha-diploma/sessions.csv"
sessions_df = pd.read_csv(sessions_csv, index_col=None, header=0)
sessions_df.drop(sessions_df.columns[sessions_df.columns.str.contains('Unnamed',case = False)],axis = 1, inplace = True)
display(sessions_df)

,0,1,2,3,4,5,6,7,8,9
0,403a42b604f504069c41d39c5a4fc2,23ee5640ec588ffaa78afa3268b04a,5b150a1cae009459493bd0fe5c7b3e,e291f8ae1de4a38c71027e9c91d51c,34cfd44ad5cc2d63441ebf437eccb5,02fbcf93a730c9c9d5e65aef41c67e,26b3f811a6b9ed75d3cc5198105fa4,6fcff414343456465369b05f6851eb,e2f15819192cded77ed1825a129c0c,41b772a5b18bc724562bfb60f4f3ac
1,a5b2dd3ed41aabb5733c7cfffff0bc,25e573b81ab69d6cf700a51281fc2a,49c58f679f72e0f02b97bdeee1a1bb,c0be492c14bcec5ab3dfd34d95d6f5,b2274b9b4c8c4dc2f79d417ddd71e5,7fd08607f52b89a753437eee177e32,25d41576f3529994c878a5d15f6dc7,ece16e15b2286f8287e8935f12b6c2,806c0ed32e135afb8db4f3aad71acf,f10b4adf1c732b6b260791fb24f430
2,80870ec50515107ba7034ab758cebe,cd522afebcf91625c3cd811f3fb6e2,061a59839837a123a916a70947f075,07e873c5aac8ffaf97897fbd0dfbf6,60fb638ff7ab469be8637111fdc837,d49c4a9b748b9c7abb9d98cdecebef,b0b062dc25c4dada316183941b100c,c7f809aab571e9acdf799a78c5ef61,67c38de267396951227897efc503e1,7b457a5d30f3cb01b6998390ec7abf
3,d66440e7524c0a8b9b206fbb7295ba,046f93c91d0befd4625f00e07eb473,199cf6d89a2f06fc552ebf86105d7d,2adfc14bbc0fba5b5d616d6fcfe3de,a458a5a362af9a7774780eb4e25365,88bbc5912f946b1fb77fdeaa3f5d4a,6d0bb6959972ac92c10d7445982a94,274869b912b256df69b9e7c5c0cb11,2c5bf20534b80ee47bed9a90b405a0,f456635bb3d2567a3a61133fa9dd89
4,231b80c884b2f1390e17f09a187889,bd6707bb04cfb8880a5f783c85be20,bbd5df907c099d2cc2b5285b08038e,514f3ee1f00df05dc3900251972245,961c336b0c2fa04c883c639dfee1ba,d6159553e41857fe23df9d3e746284,d2cd45a49d53702df403cffddd53e6,c0314a446fbef65ce9626b037a70da,d6e8f6d21631149a90ac940d3ee227,e762f208419b7449c0731037262bb3
5,d50faca942c93b02863dc4cc86a839,2530d7a6ecf471f695a2ef7a8a4191,9914764b34cff2aac25a82a33d5494,914880359e59dbe7b7e60c4ddc82f4,6fcff414343456465369b05f6851eb,41b772a5b18bc724562bfb60f4f3ac,34cfd44ad5cc2d63441ebf437eccb5,e291f8ae1de4a38c71027e9c91d51c,5b150a1cae009459493bd0fe5c7b3e,02fbcf93a730c9c9d5e65aef41c67e
6,5bc1c8052fb210cff1aabade8dec0c,3eb635e807cc6a8484531b00486f71,8c7ff3370563e8ad6e2fb534a1ede7,70a8a9dce94fa4e57d84c2ee735b7f,7d20e50d5f9a014a6078c92c5f17ad,05c92546aa14ea6148088b7c713be4,ff72d38f6672324e852d2cd1e089b7,e2791858e10f8f17bbb8ceb8d7e86b,d4e75ec0b19bc86bc148d32094633e,159a6475dde4d3afbb6447326500f8
7,5b150a1cae009459493bd0fe5c7b3e,e291f8ae1de4a38c71027e9c91d51c,34cfd44ad5cc2d63441ebf437eccb5,02fbcf93a730c9c9d5e65aef41c67e,26b3f811a6b9ed75d3cc5198105fa4,6fcff414343456465369b05f6851eb,e2f15819192cded77ed1825a129c0c,41b772a5b18bc724562bfb60f4f3ac,ee17bd77e91e5fb88c5610d0551d49,e3e5642e76f638dd918e718dfefe3a
8,2adfc14bbc0fba5b5d616d6fcfe3de,046f93c91d0befd4625f00e07eb473,a458a5a362af9a7774780eb4e25365,88bbc5912f946b1fb77fdeaa3f5d4a,6d0bb6959972ac92c10d7445982a94,274869b912b256df69b9e7c5c0cb11,2c5bf20534b80ee47bed9a90b405a0,f456635bb3d2567a3a61133fa9dd89,294b1604461720bdf36de6d46d96b3,227e9faf3bd31e0f87eb4afd72a69d
9,e291f8ae1de4a38c71027e9c91d51c,5b150a1cae009459493bd0fe5c7b3e,34cfd44ad5cc2d63441ebf437eccb5,02fbcf93a730c9c9d5e65aef41c67e,26b3f811a6b9ed75d3cc5198105fa4,6fcff414343456465369b05f6851eb,e2f15819192cded77ed1825a129c0c,41b772a5b18bc724562bfb60f4f3ac,ee17bd77e91e5fb88c5610d0551d49,e3e5642e76f638dd918e718dfefe3a


In [87]:
movies_id = sessions_df.to_numpy()
movies_id = np.array(movies_id).flatten()
print(movies_id)
train_movies_id = movies_id[:30]

print(len(np.unique(train_movies_id)))

['403a42b604f504069c41d39c5a4fc2' '23ee5640ec588ffaa78afa3268b04a'
 '5b150a1cae009459493bd0fe5c7b3e' 'e291f8ae1de4a38c71027e9c91d51c'
 '34cfd44ad5cc2d63441ebf437eccb5' '02fbcf93a730c9c9d5e65aef41c67e'
 '26b3f811a6b9ed75d3cc5198105fa4' '6fcff414343456465369b05f6851eb'
 'e2f15819192cded77ed1825a129c0c' '41b772a5b18bc724562bfb60f4f3ac'
 'a5b2dd3ed41aabb5733c7cfffff0bc' '25e573b81ab69d6cf700a51281fc2a'
 '49c58f679f72e0f02b97bdeee1a1bb' 'c0be492c14bcec5ab3dfd34d95d6f5'
 'b2274b9b4c8c4dc2f79d417ddd71e5' '7fd08607f52b89a753437eee177e32'
 '25d41576f3529994c878a5d15f6dc7' 'ece16e15b2286f8287e8935f12b6c2'
 '806c0ed32e135afb8db4f3aad71acf' 'f10b4adf1c732b6b260791fb24f430'
 '80870ec50515107ba7034ab758cebe' 'cd522afebcf91625c3cd811f3fb6e2'
 '061a59839837a123a916a70947f075' '07e873c5aac8ffaf97897fbd0dfbf6'
 '60fb638ff7ab469be8637111fdc837' 'd49c4a9b748b9c7abb9d98cdecebef'
 'b0b062dc25c4dada316183941b100c' 'c7f809aab571e9acdf799a78c5ef61'
 '67c38de267396951227897efc503e1' '7b457a5d30f3cb01b6998390ec7

In [4]:
full_session_data = {}
for i in range(0, len(trailers_df)):
    unnamed_values = []
    # unnamed_values.append(trailers_df.iloc[i]['title_x'])
    unnamed_values.append(trailers_df.iloc[i]['imdb_rating'])
    unnamed_values.append(json.loads(trailers_df.iloc[i]['emotions'].replace("\'", "\""))['negative'])
    unnamed_values.append(json.loads(trailers_df.iloc[i]['emotions'].replace("\'", "\""))['positive'])
    unnamed_values.append(json.loads(trailers_df.iloc[i]['emotions'].replace("\'", "\""))['neutral'])
    unnamed_values.append(json.loads(trailers_df.iloc[i]['Place of act'].replace("\'", "\""))[0])
    unnamed_values.append(json.loads(trailers_df.iloc[i]['Qualities'].replace("\'", "\""))[0])
    unnamed_values.append(json.loads(trailers_df.iloc[i]['Time of act'].replace("\'", "\""))[0])
    unnamed_values.append(json.loads(trailers_df.iloc[i]['Based on'].replace("\'", "\""))[0])
    unnamed_values.append(json.loads(trailers_df.iloc[i]['Mood'].replace("\'", "\""))[0])
    unnamed_values.append(json.loads(trailers_df.iloc[i]['Subgenre'].replace("\'", "\""))[0])
    unnamed_values.append(json.loads(trailers_df.iloc[i]['About'].replace("\'", "\""))[0])
    unnamed_values.append(json.loads(trailers_df.iloc[i]['Theme'].replace("\'", "\""))[0])
    
    full_session_data[trailers_df.iloc[i]['trailers_name']] = unnamed_values

In [5]:
# данные по сессиям с информацией о фильмах
session_data = []
numpy_sessions = sessions_df.to_numpy()
for i in range(0, len(numpy_sessions)):
    col = []
    for j in range(0, len(numpy_sessions[i])):
        col.append(full_session_data[numpy_sessions[i][j]])
    session_data.append(col)

print(session_data)
print(np.array(session_data).shape)

[[[5.4, 0.1778208613395691, 0.051855459809303284, 0.5312193632125854, 'США', 'дебютный', '21-й век', 'оригинального сценария', 'грустный', 'Драмы', 'преследования', 'развлекательные'], [6.0, 0.1520422399044037, 0.04743587225675583, 0.5621865391731262, 'США', 'остросюжетные', '21-й век', 'оригинального сценария', 'напряженные', 'Фантастические', 'расследование', 'остросоциальные'], [8.4, 0.19194278120994568, 0.08036746829748154, 0.5775054097175598, 'США', 'голливудские', '21-й век', 'комиксов', 'динамичный', 'Фантастические', 'героизм', 'развлекательные'], [6.2, 0.1778208613395691, 0.04743587225675583, 0.59267657995224, 'Мексика', 'голливудские', '21-й век', 'не определено', 'напряженные', 'Фантастические', 'перестрелки', 'развлекательные'], [6.3, 0.19194278120994568, 0.06009664759039879, 0.6150978803634644, 'параллельный мир', 'зрелищные', '21-й век', 'оригинального сценария', 'динамичный', 'Фантастические', 'аварии', 'философские'], [5.4, 0.16027602553367615, 0.05034062638878822, 0.60

In [6]:
train = session_data[:30]
test = session_data[30:]

In [7]:
tokenizer = tf.keras.preprocessing.text.Tokenizer(
    num_words=None,
    filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n',
    lower=True, char_level=False,
)

In [8]:
def tokenize(lang):
    lang_tokenizer = tf.keras.preprocessing.text.Tokenizer(filters='')
    lang_tokenizer.fit_on_texts(lang)
    tensor = lang_tokenizer.texts_to_sequences(lang)
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor,
                                                         padding='post')
    return tensor, lang_tokenizer

In [10]:
vocab = []
def getVocab(smth):
    for i in range(len(smth)):
        for j in range(len(smth[i])):
            for x in range(len(smth[i][j])):
                if isinstance(smth[i][j][x], str) and smth[i][j][x] not in vocab:
                    vocab.append(smth[i][j][x])



new_train = getVocab(train)

print(vocab)

['США', 'дебютный', '21-й век', 'оригинального сценария', 'грустный', 'Драмы', 'преследования', 'развлекательные', 'остросюжетные', 'напряженные', 'Фантастические', 'расследование', 'остросоциальные', 'голливудские', 'комиксов', 'динамичный', 'героизм', 'Мексика', 'не определено', 'перестрелки', 'параллельный мир', 'зрелищные', 'аварии', 'философские', 'большой город', 'костюмный', '19-й век', 'Нуар', 'трешовые', 'роман', 'пошлое', 'черная комедия', 'ограбления', 'от знаменитых продюсеров', '20-й век', 'реальных событий', 'Биографии', 'друзья', 'Россия', 'жанровый', 'ужасы', 'поездки', 'деревня', 'новейшее время', 'депрессивный', 'психологический', 'захватывающие', 'психотерапевты', 'трогательный', 'мечи и шпаги', 'Япония', 'красочный', 'душещипательные', 'аниме', 'смерть', 'СССР', 'народные', 'героический', 'исторические', 'Великая Отечественная война', 'Россия, Москва', 'с популярными актерами', 'душевные', 'отношения родителей и детей', 'жестокие', 'триллер', 'киллеры', 'сказка', 'д

In [11]:
def flatten(l): return flatten(l[0]) + (flatten(l[1:]) if len(l) > 1 else []) if type(l) is list else [l]

def vocab_creater(arr):
    flat_arr = flatten(arr)
    dic = list(dict.fromkeys(flat_arr))
    word2idx = {}
    idx2word = {}
    for i in range(len(dic)):
        word2idx[dic[i]] = i
        idx2word[i] = dic[i]

    return word2idx, idx2word, len(dic)

In [12]:
word2idx, idx2word, _ = vocab_creater(vocab)

In [13]:
vocab = []
def toArray(smth):
    new = []
    
    for i in range(len(smth)):
        i_new = []
        for j in range(len(smth[i])):
            j_new = []
            for x in range(len(smth[i][j])):
                if isinstance(smth[i][j][x], str):
                    j_new.append(float(word2idx[smth[i][j][x]]))
                else:
                    j_new.append(float(smth[i][j][x]))
            i_new.append(j_new)
        new.append(i_new)
    
    return new


new_train = toArray(train)

In [14]:
print(new_train)

[[[5.4, 0.1778208613395691, 0.051855459809303284, 0.5312193632125854, 0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0], [6.0, 0.1520422399044037, 0.04743587225675583, 0.5621865391731262, 0.0, 8.0, 2.0, 3.0, 9.0, 10.0, 11.0, 12.0], [8.4, 0.19194278120994568, 0.08036746829748154, 0.5775054097175598, 0.0, 13.0, 2.0, 14.0, 15.0, 10.0, 16.0, 7.0], [6.2, 0.1778208613395691, 0.04743587225675583, 0.59267657995224, 17.0, 13.0, 2.0, 18.0, 9.0, 10.0, 19.0, 7.0], [6.3, 0.19194278120994568, 0.06009664759039879, 0.6150978803634644, 20.0, 21.0, 2.0, 3.0, 15.0, 10.0, 22.0, 23.0], [5.4, 0.16027602553367615, 0.05034062638878822, 0.607673168182373, 24.0, 25.0, 26.0, 3.0, 9.0, 27.0, 11.0, 7.0], [4.3, 0.1732981950044632, 0.024433093145489693, 0.607673168182373, 0.0, 28.0, 2.0, 29.0, 30.0, 31.0, 32.0, 7.0], [7.5, 0.16886694729328156, 0.061885979026556015, 0.6442351341247559, 24.0, 33.0, 34.0, 35.0, 4.0, 36.0, 37.0, 12.0], [5.4, 0.14415885508060455, 0.09010298550128937, 0.607673168182373, 38.0, 39.0, 2.0, 3.0, 9.0, 4

In [16]:
scaler = MinMaxScaler((0, 1))
normalized = []
new_train = np.nan_to_num(new_train)
for i in range(len(new_train)):
    normalized.append(scaler.fit_transform(new_train[i]))

In [17]:
print(normalized)

[array([[0.26829268, 0.70446296, 0.41757898, 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        ],
       [0.41463415, 0.16497985, 0.35027892, 0.27400756, 0.        ,
        0.18421053, 0.        , 0.        , 0.125     , 0.14285714,
        0.14285714, 0.13157895],
       [1.        , 1.        , 0.85175068, 0.40955387, 0.        ,
        0.31578947, 0.        , 0.34375   , 0.275     , 0.14285714,
        0.28571429, 0.        ],
       [0.46341463, 0.70446296, 0.35027892, 0.54379328, 0.4047619 ,
        0.31578947, 0.        , 0.46875   , 0.125     , 0.14285714,
        0.37142857, 0.        ],
       [0.48780488, 1.        , 0.54307314, 0.74218418, 0.47619048,
        0.52631579, 0.        , 0.        , 0.275     , 0.14285714,
        0.45714286, 0.42105263],
       [0.26829268, 0.33729272, 0.39451158, 0.67648793, 0.57142857,
        0.63157895, 0.58536585, 0.        , 0.125     , 0.62857143,
        0.14285714

In [18]:
normalized_fl = []
for i in range(len(normalized)):
    for j in range(len(normalized[i])):
        normalized_fl.append(normalized[i][j])

In [19]:
print(np.array(normalized_fl).shape)

(300, 12)


In [20]:
print(np.array(session_data).shape)

(51, 10, 12)


In [63]:
movies_id_dataset = tf.data.Dataset.from_tensor_slices(train_movies_id)
print(movies_id_dataset)

<TensorSliceDataset shapes: (), types: tf.string>


In [60]:
dataset = tf.data.Dataset.from_tensor_slices(normalized_fl)
print(dataset)

<TensorSliceDataset shapes: (12,), types: tf.float64>


In [64]:
full_ds = tf.data.Dataset.zip((movies_id_dataset, dataset))
print(full_ds)

<ZipDataset shapes: ((), (12,)), types: (tf.string, tf.float64)>


In [65]:
seq_length = 3
examples_per_epoch = 300 // (seq_length+1)

In [79]:
sequences = dataset.batch(seq_length+1, drop_remainder=True)

for seq in sequences.take(1):
    print(seq)

tf.Tensor(
[[0.26829268 0.70446296 0.41757898 0.         0.         0.
  0.         0.         0.         0.         0.         0.        ]
 [0.41463415 0.16497985 0.35027892 0.27400756 0.         0.18421053
  0.         0.         0.125      0.14285714 0.14285714 0.13157895]
 [1.         1.         0.85175068 0.40955387 0.         0.31578947
  0.         0.34375    0.275      0.14285714 0.28571429 0.        ]
 [0.46341463 0.70446296 0.35027892 0.54379328 0.4047619  0.31578947
  0.         0.46875    0.125      0.14285714 0.37142857 0.        ]], shape=(4, 12), dtype=float64)


In [80]:
ids = movies_id_dataset.batch(seq_length+1, drop_remainder=True)
for _id in ids.take(1):
    print(seq)

tf.Tensor(
[[0.26829268 0.70446296 0.41757898 0.         0.         0.
  0.         0.         0.         0.         0.         0.        ]
 [0.41463415 0.16497985 0.35027892 0.27400756 0.         0.18421053
  0.         0.         0.125      0.14285714 0.14285714 0.13157895]
 [1.         1.         0.85175068 0.40955387 0.         0.31578947
  0.         0.34375    0.275      0.14285714 0.28571429 0.        ]
 [0.46341463 0.70446296 0.35027892 0.54379328 0.4047619  0.31578947
  0.         0.46875    0.125      0.14285714 0.37142857 0.        ]], shape=(4, 12), dtype=float64)


In [83]:
def split_input_target(sequence):
    input_text = sequence[:-1]
    target_text = sequence[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)
id_dataset = ids.map(split_input_target)

for input_example, target_example in zip(id_dataset.take(1), dataset.take(1)):
    print("Input :", input_example)
    print("Target:", target_example)

Input : (<tf.Tensor: shape=(3,), dtype=string, numpy=
array([b'403a42b604f504069c41d39c5a4fc2',
       b'23ee5640ec588ffaa78afa3268b04a',
       b'5b150a1cae009459493bd0fe5c7b3e'], dtype=object)>, <tf.Tensor: shape=(3,), dtype=string, numpy=
array([b'23ee5640ec588ffaa78afa3268b04a',
       b'5b150a1cae009459493bd0fe5c7b3e',
       b'e291f8ae1de4a38c71027e9c91d51c'], dtype=object)>)
Target: (<tf.Tensor: shape=(3, 12), dtype=float64, numpy=
array([[0.26829268, 0.70446296, 0.41757898, 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        ],
       [0.41463415, 0.16497985, 0.35027892, 0.27400756, 0.        ,
        0.18421053, 0.        , 0.        , 0.125     , 0.14285714,
        0.14285714, 0.13157895],
       [1.        , 1.        , 0.85175068, 0.40955387, 0.        ,
        0.31578947, 0.        , 0.34375   , 0.275     , 0.14285714,
        0.28571429, 0.        ]])>, <tf.Tensor: shape=(3, 12), dtype=float64, numpy

In [109]:
class MyModel(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, rnn_units):
        super().__init__(self)
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.gru = tf.keras.layers.GRU(rnn_units,
                                       return_sequences=True,
                                       return_state=True)
        self.dense = tf.keras.layers.Dense(vocab_size)
    
    def call(self, inputs_ids, inputs_data, states=None, return_state=False, training=False):
        print(inputs_ids)
        print(inputs_data)
#         ids = tf.keras.layers.Input(name="trailer_id", shape=(1,), dtype=tf.string), # 30 уникальных
#         trailers_data = tf.keras.layers.Input(name="trailers_data", shape=(12,), dtype=tf.float32),
        
#         print(ids)
#         print(trailers_data)

        if states is None:
            states = self.gru.get_initial_state(x)
        x, states = self.gru(x, initial_state=states, training=training)
        x = self.dense(x, training=training)

        if return_state:
            return x, states
        else:
            return x

In [110]:
model = MyModel(977, 1024, 3)

In [112]:
for ids, data in zip(id_dataset.take(1), dataset.take(1)):
    example_batch_predictions = model(ids[0], data[0])
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

tf.Tensor(
[b'403a42b604f504069c41d39c5a4fc2' b'23ee5640ec588ffaa78afa3268b04a'
 b'5b150a1cae009459493bd0fe5c7b3e'], shape=(3,), dtype=string)
tf.Tensor(
[[0.26829268 0.70446296 0.41757898 0.         0.         0.
  0.         0.         0.         0.         0.         0.        ]
 [0.41463415 0.16497985 0.35027892 0.27400756 0.         0.18421053
  0.         0.         0.125      0.14285714 0.14285714 0.13157895]
 [1.         1.         0.85175068 0.40955387 0.         0.31578947
  0.         0.34375    0.275      0.14285714 0.28571429 0.        ]], shape=(3, 12), dtype=float64)
(<KerasTensor: shape=(None, 1) dtype=string (created by layer 'trailer_id')>,)
(<KerasTensor: shape=(None, 12) dtype=float32 (created by layer 'trailers_data')>,)


UnboundLocalError: local variable 'x' referenced before assignment

In [42]:
model.summary()

Model: "my_model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      multiple                  1000448   
_________________________________________________________________
gru_1 (GRU)                  multiple                  9261      
_________________________________________________________________
dense_1 (Dense)              multiple                  3908      
Total params: 1,013,617
Trainable params: 1,013,617
Non-trainable params: 0
_________________________________________________________________


In [43]:
loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)

In [44]:
example_batch_loss = loss(target_example_batch, example_batch_predictions)
mean_loss = example_batch_loss.numpy().mean()
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("Mean loss:        ", mean_loss)


ValueError: Shape mismatch: The shape of labels (received (36,)) should equal the shape of logits except for the last dimension (received (3, 977)).

In [45]:
model.compile(optimizer='adam', loss=loss)

In [37]:
EPOCHS = 5
history = model.fit(dataset, epochs=EPOCHS)

Epoch 1/5
75/75 [==============================] - 2s 9ms/step - loss: 0.4275
Epoch 2/5
75/75 [==============================] - 1s 8ms/step - loss: 0.4033
Epoch 3/5
75/75 [==============================] - 1s 7ms/step - loss: 0.3902
Epoch 4/5
75/75 [==============================] - 1s 7ms/step - loss: 0.3820
Epoch 5/5
75/75 [==============================] - 1s 7ms/step - loss: 0.3764


In [38]:
scalar = 0
for input_example_batch, target_example_batch in dataset.take(30):
    prediction = model.predict(input_example_batch)
    mean = []
    for i in range(len(prediction[0])):
        mean.append(np.mean(prediction[0][i]))
    print(mean)
    
#     for i in range(len(prediction[0])):
#         scalar = np.dot(prediction[0])
#         if (max_scalar < scalar):
#             max_scalar = scalar
#             max_scalar_ind = i
        
# print(max_scalar_ind)

[-5.4939594, -5.4939656, -5.4939656, -5.4939656, -5.4939656, -5.4939656, -5.4939656, -5.4939656, -5.4939656, -5.4939656, -5.4939656, -5.4939656]
[-5.4939594, -5.4939656, -5.4939656, -5.4939656, -5.4939656, -5.4939656, -5.4939656, -5.4939656, -5.4939656, -5.4939656, -5.4939656, -5.4939656]
[-5.4939594, -5.4939656, -5.4939656, -5.4939656, -5.4939656, -5.4939656, -5.4939656, -5.4939656, -5.4939656, -5.4939656, -5.4939656, -5.4939656]
[-5.4939456, -5.4939656, -5.4939656, -5.4939656, -5.4939656, -5.4939656, -5.4939656, -5.4939656, -5.4939656, -5.4939656, -5.4939656, -5.4939656]
[-5.4939594, -5.4939656, -5.4939656, -5.4939656, -5.4939656, -5.4939656, -5.4939656, -5.4939656, -5.4939656, -5.4939656, -5.4939656, -5.4939656]
[-5.4939594, -5.4939656, -5.4939656, -5.4939656, -5.4939656, -5.4939656, -5.4939656, -5.4939656, -5.4939656, -5.4939656, -5.4939656, -5.4939656]
[-5.4939594, -5.4939656, -5.4939656, -5.4939656, -5.4939656, -5.4939656, -5.4939656, -5.4939656, -5.4939656, -5.4939656, -5.493965

In [ ]:
for input_example_batch, target_example_batch in dataset.take(1):
    print(target_example_batch.numpy())